# Introduction

A pivot table is a summarized form of a more extensive table. In simple terms, it's a breakdown of larger values. Overall, it is an essential tool for every data scientist and with the knowledge of building blocks of Pandas, it is much easier to learn. 

In [1]:
# Let's load a simple file
import pandas as pd
import numpy as np

df = pd.read_csv('./titanic/train.csv')
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
739,740,0,3,"Nankoff, Mr. Minko",male,NaN,0,0,349218,7.8958,NaN,S
99,100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S
406,407,0,3,"Widegren, Mr. Carl/Charles Peter",male,51.0,0,0,347064,7.7500,NaN,S
307,308,1,1,"Penasco y Castellana, Mrs. Victor de Satode (M...",female,17.0,1,0,PC 17758,108.9000,C65,C


With this data, if we need to analyze count of passengers by their class and break that into survived and non-survived, a pivot operation would be much easier to do this then any other method.

In [2]:
df.pivot_table(index = 'Pclass', columns = 'Survived', values = 'PassengerId', aggfunc = 'count')

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


If you go through Pandas source code, it will help you to understand that pivot_table does the same operation as the function below. 

In [3]:
df.groupby(['Pclass', 'Survived'])['PassengerId'].nunique().unstack()

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


Pivot_table groups the data first to perform the specified operation and then unstacks the result to create multi-index columns combined in a table.

# Missing value imputation

This function has fill_value parameter which will show the provided value when a missing value is generated after aggregation.

In [4]:
df.pivot_table(index = 'Pclass', columns = 'Embarked', values = 'Age', aggfunc = np.mean, fill_value = 0)

Embarked,C,Q,S
Pclass,,,
1,38.027027,38.5000,38.152037
2,22.766667,43.5000,30.386731
3,20.741951,25.9375,25.696552


# Multiple Statistics

In data analysis many times need comes to generate multiple statistisc in order to fully understand full nature of distributions and relationships.

It is very easy to do this in pivot_table, simply pass all aggregation functions in form of a list.

In [5]:
df.pivot_table(index = 'Pclass', columns = 'Embarked', values = 'Age', aggfunc = [np.mean, np.median, np.std], fill_value = 0)

mean                     median                  std  \
Embarked          C        Q          S      C     Q   S          C   
Pclass                                                                
1         38.027027  38.5000  38.152037   36.5  38.5  37  14.243454   
2         22.766667  43.5000  30.386731   25.0  43.5  30  10.192551   
3         20.741951  25.9375  25.696552   20.0  21.5  25  11.712367   

                                
Embarked          Q          S  
Pclass                          
1          7.778175  15.315584  
2         19.091883  14.080001  
3         16.807938  12.110906

# Reshaping by Stacking and Unstacking

For the previous example, after pivot operation, we may still want the data to be in rows instead of columns. In some specific cases, it's much easier to handle a long format data than broad data. Pandas can easily do reshaping of these new dataframes and create a series of values stacked by the values in columns and index.

In [6]:
df2 = df.pivot_table(index = 'Pclass', columns = 'Embarked', values = 'Age', aggfunc = np.median)
stacked = df2.stack()
stacked

Pclass  Embarked
1       C           36.5
        Q           38.5
        S           37.0
2       C           25.0
        Q           43.5
        S           30.0
3       C           20.0
        Q           21.5
        S           25.0
dtype: float64

Also, it's pretty easy to run an inverse operation to get the pivoted dataframe.

In [7]:
unstacked = stacked.unstack()
unstacked

Embarked,C,Q,S
Pclass,,,
1,36.5,38.5,37.0
2,25.0,43.5,30.0
3,20.0,21.5,25.0


While performing the reverse operation, you may like to change the columns with rows and vice-versa. For this simply pass the order of the index in unstack function call.

In [8]:
unstacked = stacked.unstack(0)
unstacked

Pclass,1,2,3
Embarked,,,
C,36.5,25.0,20.0
Q,38.5,43.5,21.5
S,37.0,30.0,25.0


# Cross tabulations

Sometimes we are just interested in knowing frequencies of observations across categorical columns. crosstab is a very easy and convenient apporach in such cases.

In [9]:
pd.crosstab(df['Embarked'], df['Pclass'])

Pclass,1,2,3
Embarked,,,
C,85,17,66
Q,2,3,72
S,127,164,353


Frequencies can also be normalized on overall frequency or on column/row frequecy.

In [10]:
## Normalized on total count
pd.crosstab(df['Embarked'], df['Pclass'], normalize = True)

Pclass,1,2,3
Embarked,,,
C,0.095613,0.019123,0.074241
Q,0.002250,0.003375,0.080990
S,0.142857,0.184477,0.397075


In [11]:
## Normalized on column count
pd.crosstab(df['Embarked'], df['Pclass'], normalize = 'columns')

Pclass,1,2,3
Embarked,,,
C,0.397196,0.092391,0.134420
Q,0.009346,0.016304,0.146640
S,0.593458,0.891304,0.718941


Just like pivot_table, crosstab can also perform aggregation. Additionally it can normalize values and add margins to end output.

In [12]:
pd.crosstab(df['Embarked'], df['Pclass'], values = df['Fare'], aggfunc = np.sum, margins = True, normalize = True)

Pclass,1,2,3,All
Embarked,,,,
C,0.311947,0.015108,0.025939,0.352993
Q,0.006308,0.001298,0.028219,0.035826
S,0.313183,0.116833,0.181165,0.611181
All,0.631438,0.133239,0.235323,1.000000


# Dummy Variables

Generation of indicator columns from categorical variable is an important operation in machine learning application. Pandas provides easy function to achieve this.

In [13]:
df_new = pd.concat([df, pd.get_dummies(df['Pclass'])], axis = 1)
df_new.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,1,2,3
384,385,0,3,"Plotcharsky, Mr. Vasil",male,NaN,0,0,349227,7.8958,NaN,S,0,0,1
435,436,1,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,B96 B98,S,1,0,0
620,621,0,3,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,0,0,1
618,619,1,2,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0000,F4,S,0,1,0
787,788,0,3,"Rice, Master. George Hugh",male,8.0,4,1,382652,29.1250,NaN,Q,0,0,1
520,521,1,1,"Perreault, Miss. Anne",female,30.0,0,0,12749,93.5000,B73,S,1,0,0
839,840,1,1,"Marechal, Mr. Pierre",male,NaN,0,0,11774,29.7000,C47,C,1,0,0
327,328,1,2,"Ball, Mrs. (Ada E Hall)",female,36.0,0,0,28551,13.0000,D,S,0,1,0
690,691,1,1,"Dick, Mr. Albert Adrian",male,31.0,1,0,17474,57.0000,B20,S,1,0,0
670,671,1,2,"Brown, Mrs. Thomas William Solomon (Elizabeth ...",female,40.0,1,1,29750,39.0000,NaN,S,0,1,0


The same function can be used to create a one-hot encoded column where first value can be dropped as a reference. 

In [14]:
df_new = pd.concat([df, pd.get_dummies(df['Pclass'], drop_first = True)], axis = 1)
df_new.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,2,3
110,111,0,1,"Porter, Mr. Walter Chamberlain",male,47.00,0,0,110465,52.0000,C110,S,0,0
247,248,1,2,"Hamalainen, Mrs. William (Anna)",female,24.00,0,2,250649,14.5000,NaN,S,1,0
833,834,0,3,"Augustsson, Mr. Albert",male,23.00,0,0,347468,7.8542,NaN,S,0,1
60,61,0,3,"Sirayanian, Mr. Orsen",male,22.00,0,0,2669,7.2292,NaN,C,0,1
198,199,1,3,"Madigan, Miss. Margaret ""Maggie""",female,NaN,0,0,370370,7.7500,NaN,Q,0,1
262,263,0,1,"Taussig, Mr. Emil",male,52.00,1,1,110413,79.6500,E67,S,0,0
190,191,1,2,"Pinsky, Mrs. (Rosa)",female,32.00,0,0,234604,13.0000,NaN,S,1,0
791,792,0,2,"Gaskell, Mr. Alfred",male,16.00,0,0,239865,26.0000,NaN,S,1,0
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C,0,1
697,698,1,3,"Mullens, Miss. Katherine ""Katie""",female,NaN,0,0,35852,7.7333,NaN,Q,0,1
